<a href="https://colab.research.google.com/github/Devaki01/ACM-Responsible-AI/blob/main/ACM_WS_GNN_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import scipy.sparse as sp

# 1. Download the dataset from the provided GitHub repo
if not os.path.exists('citeseer-dataset'):
    !git clone https://github.com/ZPowerZ/citeseer-dataset.git

# 2. Data Loading and Preprocessing Functions
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)), dtype=np.int32)
    return labels_onehot

def normalize_adj(mx):
    """Row-normalize sparse matrix (Laplacian normalization): D^-0.5 * A * D^-0.5"""
    rowsum = np.array(mx.sum(1))
    r_inv_sqrt = np.power(rowsum, -0.5).flatten()
    r_inv_sqrt[np.isinf(r_inv_sqrt)] = 0.
    r_mat_inv_sqrt = sp.diags(r_inv_sqrt)
    return mx.dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt)

def load_data():
    print("Loading CiteSeer dataset...")
    path = "citeseer-dataset/"

    # Load content file: <paper_id> <word_attributes...> <class_label>
    idx_features_labels = np.genfromtxt(f"{path}citeseer.content", dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    labels = encode_onehot(idx_features_labels[:, -1])

    # Build graph
    idx = np.array(idx_features_labels[:, 0], dtype=np.dtype(str))
    idx_map = {j: i for i, j in enumerate(idx)}

    # Load cites file: <cited_id> <citing_id>
    edges_unordered = np.genfromtxt(f"{path}citeseer.cites", dtype=np.dtype(str))

    # Filter edges where both nodes are in the content file
    edges = []
    for edge in edges_unordered:
        if edge[0] in idx_map and edge[1] in idx_map:
            edges.append([idx_map[edge[0]], idx_map[edge[1]]])
    edges = np.array(edges)

    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]), dtype=np.float32)

    # Build symmetric adjacency matrix
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    # Add self-loops and normalize
    adj = normalize_adj(adj + sp.eye(adj.shape[0]))

    # Convert to Tensors
    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])
    adj = torch.FloatTensor(np.array(adj.todense()))

    # Create random masks for Train/Val/Test
    idx_rand = torch.randperm(len(labels))
    train_mask = idx_rand[:200]
    val_mask = idx_rand[200:500]
    test_mask = idx_rand[500:1500]

    return adj, features, labels, train_mask, val_mask, test_mask

# 3. Manual GCN Layer Implementation
class GraphConvolution(nn.Module):
    """
    Simple GCN layer: Z = A_norm * X * W + b
    """
    def __init__(self, in_features, out_features):
        super(GraphConvolution, self).__init__()
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        self.bias = nn.Parameter(torch.FloatTensor(out_features))
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / np.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        # 1. Feature transformation (X * W)
        support = torch.mm(input, self.weight)
        # 2. Message aggregation (A_norm * support)
        output = torch.mm(adj, support)
        return output + self.bias

# 4. GCN Model
class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()
        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        self.dropout = dropout

    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        return F.log_softmax(x, dim=1)

# 5. Training Setup
adj, features, labels, train_idx, val_idx, test_idx = load_data()

model = GCN(nfeat=features.shape[1],
            nhid=16,
            nclass=labels.max().item() + 1,
            dropout=0.5)

optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

# 6. Training Loop
print("\nStarting Training...")
for epoch in range(200):
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[train_idx], labels[train_idx])
    acc_train = accuracy(output[train_idx], labels[train_idx])
    loss_train.backward()
    optimizer.step()

    if epoch % 20 == 0:
        model.eval()
        output = model(features, adj)
        loss_val = F.nll_loss(output[val_idx], labels[val_idx])
        acc_val = accuracy(output[val_idx], labels[val_idx])
        print(f'Epoch: {epoch:04d} | Loss: {loss_train.item():.4f} | Acc: {acc_train.item():.4f} | Val Acc: {acc_val.item():.4f}')

# 7. Testing
model.eval()
output = model(features, adj)
loss_test = F.nll_loss(output[test_idx], labels[test_idx])
acc_test = accuracy(output[test_idx], labels[test_idx])
print(f"\nTest set results: Loss = {loss_test.item():.4f}, Accuracy = {acc_test.item():.4f}")

Cloning into 'citeseer-dataset'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 12 (delta 1), reused 12 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (12/12), 347.81 KiB | 1014.00 KiB/s, done.
Resolving deltas: 100% (1/1), done.
Loading CiteSeer dataset...

Starting Training...
Epoch: 0000 | Loss: 1.9790 | Acc: 0.1600 | Val Acc: 0.1900
Epoch: 0020 | Loss: 0.2169 | Acc: 0.9400 | Val Acc: 0.6533
Epoch: 0040 | Loss: 0.0895 | Acc: 0.9800 | Val Acc: 0.6800
Epoch: 0060 | Loss: 0.0703 | Acc: 0.9900 | Val Acc: 0.6833
Epoch: 0080 | Loss: 0.0428 | Acc: 0.9950 | Val Acc: 0.6867
Epoch: 0100 | Loss: 0.0501 | Acc: 0.9900 | Val Acc: 0.6867
Epoch: 0120 | Loss: 0.0442 | Acc: 0.9850 | Val Acc: 0.6867
Epoch: 0140 | Loss: 0.0406 | Acc: 0.9950 | Val Acc: 0.6867
Epoch: 0160 | Loss: 0.0379 | Acc: 0.9900 | Val Acc: 0.6900
Epoch: 0180 | Loss: 0.0499 | Acc: 0.9850 | Val Acc: 0.6967

Test set results:

In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import scipy.sparse as sp

# 1. Setup and Data Loading (Consistent with previous step)
if not os.path.exists('citeseer-dataset'):
    !git clone https://github.com/ZPowerZ/citeseer-dataset.git

def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in enumerate(classes)}
    return np.array(list(map(classes_dict.get, labels)), dtype=np.int32)

def normalize_adj(mx):
    """Row-normalize sparse matrix: D^-0.5 * A * D^-0.5"""
    rowsum = np.array(mx.sum(1))
    r_inv_sqrt = np.power(rowsum, -0.5).flatten()
    r_inv_sqrt[np.isinf(r_inv_sqrt)] = 0.
    r_mat_inv_sqrt = sp.diags(r_inv_sqrt)
    return mx.dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt)

def get_processed_data(use_self_loops=True):
    path = "citeseer-dataset/"
    idx_features_labels = np.genfromtxt(f"{path}citeseer.content", dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    labels = encode_onehot(idx_features_labels[:, -1])
    idx = np.array(idx_features_labels[:, 0], dtype=np.dtype(str))
    idx_map = {j: i for i, j in enumerate(idx)}
    edges_unordered = np.genfromtxt(f"{path}citeseer.cites", dtype=np.dtype(str))

    edges = np.array([[idx_map[e[0]], idx_map[e[1]]] for e in edges_unordered if e[0] in idx_map and e[1] in idx_map])
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]), dtype=np.float32)

    # Symmetric adjacency
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    # TOGGLE SELF-LOOPS HERE
    if use_self_loops:
        adj = adj + sp.eye(adj.shape[0])

    adj = normalize_adj(adj)

    # Convert to tensors
    features_t = torch.FloatTensor(np.array(features.todense()))
    labels_t = torch.LongTensor(np.where(labels)[1])
    adj_t = torch.FloatTensor(np.array(adj.todense()))

    return adj_t, features_t, labels_t

# 2. GCN Architecture
class GraphConvolution(nn.Module):
    def __init__(self, in_features, out_features):
        super(GraphConvolution, self).__init__()
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        self.bias = nn.Parameter(torch.FloatTensor(out_features))
        nn.init.xavier_uniform_(self.weight)
        nn.init.zeros_(self.bias)

    def forward(self, x, adj):
        return torch.mm(adj, torch.mm(x, self.weight)) + self.bias

class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass):
        super(GCN, self).__init__()
        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)

    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, 0.5, training=self.training)
        return F.log_softmax(self.gc2(x, adj), dim=1)

# 3. Training Function
def train_model(adj, features, labels, name):
    print(f"\n--- Training GCN ({name}) ---")
    idx_rand = torch.randperm(len(labels))
    train_idx, test_idx = idx_rand[:200], idx_rand[500:1500]

    model = GCN(features.shape[1], 16, labels.max().item() + 1)
    optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

    best_acc = 0
    for epoch in range(100):
        model.train()
        optimizer.zero_grad()
        output = model(features, adj)
        loss = F.nll_loss(output[train_idx], labels[train_idx])
        loss.backward()
        optimizer.step()

        if epoch % 20 == 0:
            model.eval()
            preds = output[test_idx].max(1)[1]
            acc = preds.eq(labels[test_idx]).double().mean()
            best_acc = max(best_acc, acc.item())
            print(f"Epoch {epoch:03d} | Loss: {loss.item():.4f} | Test Acc: {acc.item():.4f}")

    return best_acc

# 4. Run Comparison
# Run with Self-Loops
adj_with, feat, labels = get_processed_data(use_self_loops=True)
acc_with = train_model(adj_with, feat, labels, "With Self-Loops")

# Run without Self-Loops
adj_without, feat, labels = get_processed_data(use_self_loops=False)
acc_without = train_model(adj_without, feat, labels, "Without Self-Loops")

print("\n" + "="*30)
print(f"FINAL COMPARISON (Accuracy)")
print(f"GCN with Self-Loops:    {acc_with:.4f}")
print(f"GCN without Self-Loops: {acc_without:.4f}")
print("="*30)


--- Training GCN (With Self-Loops) ---
Epoch 000 | Loss: 1.8062 | Test Acc: 0.1260
Epoch 020 | Loss: 0.1342 | Test Acc: 0.6180
Epoch 040 | Loss: 0.0623 | Test Acc: 0.6160
Epoch 060 | Loss: 0.0542 | Test Acc: 0.6200
Epoch 080 | Loss: 0.0768 | Test Acc: 0.6220

--- Training GCN (Without Self-Loops) ---
Epoch 000 | Loss: 1.7992 | Test Acc: 0.1930
Epoch 020 | Loss: 0.2187 | Test Acc: 0.5900
Epoch 040 | Loss: 0.1697 | Test Acc: 0.5710
Epoch 060 | Loss: 0.1016 | Test Acc: 0.5980
Epoch 080 | Loss: 0.1158 | Test Acc: 0.6050

FINAL COMPARISON (Accuracy)
GCN with Self-Loops:    0.6220
GCN without Self-Loops: 0.6050
